## RAG example with Langchain, Redis, and HFTGI

Requirements:
- A Redis cluster and Database where documents have been injected
- All information for connecting to the redis cluster and database, index name and schema file.
- An inference endpoint served with Hugging Face Text Generation Inference server

#### Bases parameters

In [1]:
index_name = 'pdf_docs'
# Define the OpenAI API key
openai_api_key = 'your_openai_api_key'

#### Imports

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.llms import OpenAI
import openai
import os

#### Initialize the connection

In [ ]:
embeddings = HuggingFaceEmbeddings()
faiss_index = FAISS.load_local(folder_path="C:/Users/User/PycharmProjects/ai-course/lession8/pdf", embeddings=embeddings, index_name=index_name, allow_dangerous_deserialization=True)

retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 4})

#### Initialize query chain

In [ ]:


# NOTE: This template syntax is specific to Llama2
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
You will be given a question you need to answer, and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Question: {question}
Context: {context} [/INST]
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Initialize OpenAI LLM
os.environ["OPENAI_API_KEY"] = openai_api_key
llm = OpenAI(model="text-davinci-003", max_tokens=512, temperature=0.1)

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)

#### Query example

In [ ]:
question = "How can I work with GPU and taints?"
result = qa_chain({"query": question})

#### Retrieve source

In [ ]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)